# 十種猴子影像辨識

## 導入模組

In [3]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, Conv, MaxPool, onecold, onehotbatch, throttle, logitcrossentropy
using Augmentor
using Statistics
using Images
using JLD

## 載入資料

In [4]:
pl = Resize(128, 128) |>
     Either(1=>FlipX(), 1=>NoOp()) |>
     Either(1=>Zoom(0.9:0.1:1.2), 1=>NoOp()) |>
     CropSize(128, 128)

4-step Augmentor.ImmutablePipeline:
 1.) Resize to 128×128
 2.) Either: (50%) Flip the X axis. (50%) No operation.
 3.) Either: (50%) Zoom by I ∈ {0.9×0.9, 1.0×1.0, 1.1×1.1, 1.2×1.2}. (50%) No operation.
 4.) Crop a 128×128 window around the center

### 載入訓練資料

In [5]:
monkeys = Array{Float64}(undef, 1097, 128, 128, 3)
mon_key = 1
pbase = "training/training/n"
for i in (0:9)
    stri = string(i)
    p = pbase * stri * "/n" * stri * " ("
    amount = [105, 111, 110, 122, 105, 113, 106, 114, 106, 105]
    for j in (1:amount[i + 1])
        imarray = []
        strj = string(j)
        path = p * strj * ").jpg"
        img = load(path)
        imgg = augment(img, pl)
        mat = channelview(imgg)
        c = copy(mat)
        c2 = zeros(128, 128, 3)
        convert(Array{N0f8, 3}, c2)
        for i in (1:3)
            for j in (1:128)
                for k in (1:128)
                    c2[j, k, i] = copy(convert(N0f8, c[i, j, k]))
                end
            end
        end
        monkeys[mon_key, :, :, :] = c2
        mon_key += 1
    end
end

### 儲存訓練資料

In [6]:
save("monkey_data.jld", "data", monkeys)

### 讀入已儲存的訓練資料

In [7]:
train_x = load("monkey_data.jld")["data"]
train_x = reshape(train_x, 128, 128, 3, :)
train_x = Float32.(train_x)

128×128×3×1097 Array{Float32,4}:
[:, :, 1, 1] =
 0.529412   0.301961    0.988235  …  0.203922   0.0588235  0.647059
 0.12549    0.168627    0.878431     0.443137   0.219608   0.505882
 0.470588   0.32549     0.380392     0.466667   0.494118   0.0313726
 0.439216   0.145098    0.32549      0.470588   0.25098    0.654902
 0.901961   0.682353    0.854902     0.262745   0.341176   0.443137
 0.423529   0.00392157  0.505882  …  0.729412   0.384314   0.67451
 0.490196   0.192157    0.745098     0.290196   0.396078   0.996078
 0.243137   0.00784314  0.654902     0.698039   0.560784   0.580392
 0.294118   0.835294    0.47451      0.117647   0.0862745  0.447059
 0.427451   0.270588    1.0          0.0196078  0.196078   0.0352941
 0.192157   0.219608    0.945098  …  0.227451   0.160784   0.4
 0.407843   0.662745    0.686275     0.258824   0.215686   0.0352941
 0.764706   0.415686    0.372549     0.262745   0.286275   0.482353
 ⋮                                ⋱  ⋮                     
 0.52549   

### 載入驗證資料

In [8]:
val_monkeys = Array{Float64}(undef, 272, 128, 128, 3)
val_key = 1
pbase = "validation/validation/n"
for i in (0:9)
    stri = string(i)
    p = pbase * stri * "/n" * stri * " ("
    amount = [26, 28, 27, 30, 26, 28, 26, 28, 27, 26]
    for j in (1:amount[i + 1])
        imarray = []
        strj = string(j)
        path = p * strj * ").jpg"
        img = load(path)
        imgg = augment(img, pl)
        mat = channelview(imgg)
        c = copy(mat)
        c2 = zeros(128, 128, 3)
        convert(Array{N0f8, 3}, c2)
        for i in (1:3)
            for j in (1:128)
                for k in (1:128)
                    c2[j, k, i] = copy(convert(N0f8, c[i, j, k]))
                end
            end
        end
        val_monkeys[val_key, :, :, :] = c2
        val_key += 1
    end
end

### 儲存驗證資料

In [9]:
save("monkey_validation.jld", "data", val_monkeys)

### 讀入已儲存的驗證資料

In [10]:
test_x = load("monkey_validation.jld")["data"]
test_x = reshape(test_x, 128, 128, 3, :)
test_x = Float32.(test_x)

128×128×3×272 Array{Float32,4}:
[:, :, 1, 1] =
 0.27451   0.384314   0.976471   0.286275  …  0.513726    0.32549    0.415686
 0.513726  0.647059   0.694118   0.121569     0.211765    0.392157   1.0
 0.898039  0.278431   0.482353   0.12549      0.501961    0.760784   0.568627
 0.121569  0.0        0.596078   0.890196     0.27451     0.894118   0.262745
 0.627451  0.258824   0.662745   0.313726     0.129412    0.807843   0.490196
 0.113725  0.635294   0.211765   0.27451   …  0.333333    0.427451   0.517647
 0.929412  0.694118   0.752941   0.305882     0.509804    0.866667   0.396078
 0.152941  0.168627   0.737255   0.176471     0.258824    0.768627   0.262745
 0.6       0.309804   0.309804   0.180392     0.541176    0.603922   0.235294
 0.564706  0.364706   0.415686   0.721569     0.137255    0.807843   0.419608
 0.772549  0.168627   0.156863   0.419608  …  0.34902     0.717647   0.211765
 0.8       0.72549    0.827451   0.254902     0.713726    0.0980392  0.27451
 0.443137  0.12549    0

### 訓練標籤

In [11]:
train_labels = Array{Int32}(undef, 1097)
key = 1
amount = [105, 111, 110, 122, 105, 113, 106, 114, 106, 105]
for i in (0:9)
    for j in (1:amount[i + 1])
        train_labels[key] = i
        key += 1
    end
end
train_y = onehotbatch(train_labels, 0:9)

10×1097 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

### 驗證標籤

In [12]:
val_labels = Array{Int32}(undef, 272)
key = 1
amount = [26, 28, 27, 30, 26, 28, 26, 28, 27, 26]
for i in (0:9)
    for j in (1:amount[i + 1])
        val_labels[key] = i
        key += 1
    end
end
test_y = onehotbatch(val_labels, 0:9)

10×272 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  1  1  1  1  1  1  1  1  1  1  1

In [13]:
batchsize = 32
train = DataLoader(train_x, train_y, batchsize = batchsize, shuffle = true)
test = DataLoader(test_x, test_y, batchsize = batchsize)

DataLoader((Float32[0.27450982 0.38431373 … 0.3254902 0.41568628; 0.5137255 0.64705884 … 0.39215687 1.0; … ; 0.18039216 0.3372549 … 0.58431375 0.101960786; 0.67058825 0.52156866 … 0.2784314 0.30980393]

Float32[0.5921569 0.09019608 … 0.06666667 0.5254902; 0.3137255 0.54901963 … 0.2 0.47058824; … ; 0.27450982 0.5803922 … 0.14509805 0.14901961; 0.46666667 0.3764706 … 0.43137255 0.49019608]

Float32[0.46666667 1.0 … 0.08627451 0.54509807; 0.9411765 0.8862745 … 1.0 0.43529412; … ; 0.41568628 0.6313726 … 0.14117648 0.54901963; 0.7529412 0.19215687 … 0.3372549 0.43529412]

Float32[0.22352941 0.20784314 … 0.5019608 0.28627452; 0.59607846 0.23921569 … 0.42352942 0.9529412; … ; 0.28627452 0.6431373 … 0.36078432 0.39607844; 0.43529412 0.7607843 … 0.54901963 0.26666668]

Float32[0.5803922 0.3137255 … 0.54901963 0.20392157; 0.99215686 0.54901963 … 0.11372549 0.29803923; … ; 0.6156863 0.627451 … 0.6156863 0.30588236; 0.34117648 0.015686275 … 0.41960785 0.89411765]

Float32[0.18431373 0.27058825 … 0

## CNN 模型

In [14]:
model_easy = Chain(
    
    BatchNorm(3),
    Conv((3, 3), 3 => 64, pad = (1, 1), relu), # 128x128
    MaxPool((2, 2), stride = (2, 2)), # 64x64

    BatchNorm(64),
    Conv((3, 3), 64 => 128, pad = (1, 1), relu), # 64x64
    MaxPool((2, 2), stride = (2, 2)), # 32x32

    BatchNorm(128),
    Conv((3, 3), 128 => 64, pad = (1, 1), relu), # 32x32
    MaxPool((2, 2), stride = (2, 2)), # 16x16

    BatchNorm(64),
    Conv((3, 3), 64 => 32, pad = (1, 1), relu), # 16x16
    MaxPool((2, 2), stride = (2, 2)), # 8x8
    
    Flux.flatten,
    Dense(2048, 256, relu),
    BatchNorm(256),
    Dense(256, 10),
    softmax
)

Chain(BatchNorm(3), Conv((3, 3), 3=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(64), Conv((3, 3), 64=>128, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(128), Conv((3, 3), 128=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), BatchNorm(64), Conv((3, 3), 64=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(2048, 256, relu), BatchNorm(256), Dense(256, 10), softmax)

In [15]:
size(train_x)

(128, 128, 3, 1097)

In [16]:
model_easy(train_x[:, :, :, 1:32])

10×32 Array{Float32,2}:
 0.0828393  0.0852622  0.0867795  …  0.0843285  0.0851511  0.0844961
 0.0998173  0.0979008  0.0966907     0.0989372  0.0991987  0.0980655
 0.109566   0.108944   0.113304      0.113546   0.111502   0.110652
 0.0905804  0.0902394  0.0891006     0.0899125  0.0889595  0.0904499
 0.111909   0.112057   0.110555      0.109968   0.110279   0.110166
 0.111078   0.114623   0.112297   …  0.113815   0.115168   0.114859
 0.097054   0.0945454  0.096406      0.0962641  0.0972826  0.096013
 0.0868948  0.0866929  0.0867126     0.0878254  0.0869591  0.0868086
 0.0935181  0.094508   0.0931859     0.0918976  0.0919756  0.093894
 0.116744   0.115227   0.114969      0.113505   0.113524   0.114597

## 損失函數

In [17]:
loss(x, y) = logitcrossentropy(model_easy(x), y)

loss (generic function with 1 method)

In [18]:
accuracy(x, y, model) = mean(onecold(model_easy(x)) .== onecold(y))

accuracy (generic function with 1 method)

## Callback 函式

In [19]:
function callback()
    print("Testing ")
    @show loss(test_x, test_y)
    print("Testing ")
    @show accuracy(test_x, test_y, model_easy)
    println()
    accuracy(test_x, test_y, model_easy) > 0.8 && Flux.stop()
end

callback (generic function with 1 method)

## 模型訓練

In [20]:
epochs = 15
@epochs epochs Flux.train!(loss, params(model_easy), train, 
                           ADAM(0.005, (0.9, 0.8)), 
                           cb = throttle(callback, 500))

┌ Info: Epoch 1
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.3015082f0
Testing accuracy(test_x, test_y, model_easy) = 0.11397058823529412



┌ Info: Epoch 2
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.3380916f0
Testing accuracy(test_x, test_y, model_easy) = 0.09191176470588236



┌ Info: Epoch 3
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


...

Testing loss(test_x, test_y) = 2.113649f0
Testing accuracy(test_x, test_y, model_easy) = 0.33088235294117646



┌ Info: Epoch 14
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.175762f0
Testing accuracy(test_x, test_y, model_easy) = 0.28308823529411764



┌ Info: Epoch 15
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


Testing loss(test_x, test_y) = 2.1062906f0
Testing accuracy(test_x, test_y, model_easy) = 0.3492647058823529



## 模型評估

In [27]:
print("Validation Accuracy: $(accuracy(test_x, test_y, model_easy) * 100)%")

Validation Accuracy: 24.264705882352942%